<a href="https://colab.research.google.com/github/Benmoussa-marouane/data-science/blob/master/fileManagement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# access data in drive
from google.colab import drive
drive.mount('/content/drive')
#cd to unet directory
%cd drive/My\ Drive/\Air-D/\Unet-5

In [0]:
## Imports
import sys
import random
import math
import os,errno
import glob
import pathlib
import shutil
import numpy as np
import copy
from collections import Counter
from keras.preprocessing.image import ImageDataGenerator



from sklearn.metrics import jaccard_score
import numpy as np
import cv2
import matplotlib.pyplot as plt
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from tensorflow import keras

#local library
# import manageFiles

# Image segmentation

In [0]:
# Install required libs
!pip  install git+https://github.com/qubvel/segmentation_models
# ! pip install opencv-python

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow_datasets as tfds
tfds.disable_progress_bar()
from IPython.display import clear_output
import matplotlib.pyplot as plt

#### Dataset of pollution 

Run this only once, it orginize your data using the local library manageFiles

In [0]:
def normalize(input_image, input_mask):
  input_image = tf.cast(input_image, tf.float32) / 255.0
  input_mask = tf.cast(input_mask, tf.float32) / 255.0
  return input_image, input_mask

@tf.function
def FlipImages(input_image, input_mask):
  
  if tf.random.uniform(()) > 0.5:
    input_image = tf.image.flip_left_right(input_image)
    input_mask = tf.image.flip_left_right(input_mask)

  input_image, input_mask = normalize(input_image, input_mask)

  return input_image, input_mask

In [0]:
# def CreateTraining(Quartier, new_path ,dstP):

# # Training Ids
#     for i in Quartier :
#         ids = next(os.walk(os.path.join(dstP, str(i))))[1]
#         # print(ids)
#         for j in ids:
#           src = os.path.join(dstP, str(i) ,str(j))
#           dst = os.path.join(new_path, "test")

# # move data to their correspondant quartier
#           shutil.move(src, dst)

# path_principale = "/content/drive/My Drive/Air-D/Unet-5/new_dataset_images"
# TrainP = os.path.join(path_principale,"Train")
# TestP = os.path.join(path_principale,"Test")

# new_path = "/content/drive/My Drive/Air-D/Unet-5/dataset_images"

# quartier_id  = next(os.walk(TestP))[1]

# # print(quartier_id)
# CreateTraining(quartier_id, new_path, TestP )


In [0]:
# Set some parameters
im_width = 128
im_height = 128
border = 5
path_train = '../input/train/'
path_test = '../input/test/'

path_principale = "/content/drive/My Drive/Air-D/Unet-5/new_dataset_images"
TrainP = os.path.join(path_principale,"Train")
TestP = os.path.join(path_principale,"Test")


# Load The Images
# Get and resize train images and masks

def get_data(path, train=True):

    ids = next(os.walk(path + "images"))[2]
    X = np.zeros((len(ids), im_height, im_width, 1), dtype=np.float32)

    if train:
        y = np.zeros((len(ids), im_height, im_width, 1), dtype=np.float32)
    print('Getting and resizing images ... ')

    for n, id_ in tqdm_notebook(enumerate(ids), total=len(ids)):
        # Load images
        img = load_img(path + '/images/' + id_, grayscale=True)
        x_img = img_to_array(img)
        x_img = resize(x_img, (128, 128, 1), mode='constant', preserve_range=True)

        # Load masks
        if train:
            mask = img_to_array(load_img(path + '/masks/' + id_, grayscale=True))
            mask = resize(mask, (128, 128, 1), mode='constant', preserve_range=True)

        # Save images
        X[n, ..., 0] = x_img.squeeze() / 255
        if train:
            y[n] = mask / 255
    print('Done!')
    if train:
        return X, y
    else:
        return X
    
X, y = get_data(path_train, train=True)

# Getting and resizing images 

# Split train and valid
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.15, random_state=2018)

# Check if training data looks all right
ix = random.randint(0, len(X_train))
has_mask = y_train[ix].max() > 0

fig, ax = plt.subplots(1, 2, figsize=(20, 10))

ax[0].imshow(X_train[ix, ..., 0], cmap='seismic', interpolation='bilinear')
if has_mask:
    ax[0].contour(y_train[ix].squeeze(), colors='k', levels=[0.5])
ax[0].set_title('Seismic')

ax[1].imshow(y_train[ix].squeeze(), interpolation='bilinear', cmap='gray')
ax[1].set_title('Salt');

In [0]:
# #########################################################################################################################################
# #                                                       Generator                                                                       #
# #########################################################################################################################################

# class DataGen(keras.utils.Sequence):

#     def __init__(self, quartier_ids, path, image_size=128,shuffle=True):
#         self.quartier_ids = quartier_ids
#         self.ids = []
#         self.path = path
#         self.image_size = image_size
#         self.shuffle = shuffle
#         self.on_epoch_end()

#     def __load__(self):
#         images_x  = []
#         images_y = []
    
#         ## Path
#         for Q in self.quartier_ids:
#             Q_path = os.path.join(self.path, Q)
#             self.ids = os.listdir(Q_path)

#             for id_name in self.ids :
#                 y_path = os.path.join(Q_path,id_name,"Y","pollutionMap") + ".png"
                
#                 ## Reading Image y
#                 image_y = cv2.imread(y_path,cv2.IMREAD_GRAYSCALE)
#                 image_y = cv2.resize(image_y,(self.image_size,self.image_size))
                
#                 ## Normalazing
#                 image_y = image_y/255.0        
#                 image_y = np.reshape(image_y,(self.image_size,self.image_size,1))
#                 ## Reading X images 
                
#                 x_path = os.path.join(Q_path,id_name,"X/")
#                 all_x_files_names = sorted(os.listdir(x_path))

#                 image_group_x=np.zeros((128,128,len(all_x_files_names)))
                
#                 for i,file_name in enumerate(all_x_files_names):

#                     path_x_file_name = os.path.join(x_path,file_name)
#                     image_x = cv2.imread(path_x_file_name,cv2.IMREAD_GRAYSCALE)
#                     image_x = cv2.resize(image_x,(128, 128))
#                     ## Normlazing
#                     image_x=image_x/255.0
#                     image_group_x[:,:,i]=image_x
#                     # save images  
#                     images_x.append(image_group_x)
#                     images_y.append(image_y)

#                     print(" quartier {0} Id name {1}".format(Q,file_name))    
        
#         images_x  = np.array(images_x)    
#         images_y = np.array(images_y)

#         print(" ")
#         print("images of training ",len(images_x))
#         print(" ")
#         print("images y ", len(images_y))

#         return images_x, images_y
    
#     def on_epoch_end(self):
#         """'Updates indexes after each epoch'"""
#         self.indexes = np.arange(len(self.ids))
#         if self.shuffle == True:
#               np.random.shuffle(self.indexes)
    
#     def __len__(self):
#         return int(np.ceil(len(self.ids)/float(self.batch_size)))

In [0]:
# #Getting testing
# Test_ids = os.listdir(TestP) 
# gen_test = DataGen(Test_ids, TestP, image_size=128)
# x_test,y_test = gen_test.__load__()
# print(x_test.shape , y_test.shape)

In [0]:
# image_size = 256
# epochs = 3
# batch_size = 6
# val_data_size = 12

# ## Training Ids

# Train_ids = os.listdir(TrainP)

# ## validation Ids 
# Test_ids = os.listdir(TestP) 

# #Getting Training
# gen = DataGen(Train_ids, TrainP, image_size=128)
# x,y = gen.__load__()
# print(x.shape , y.shape)

In [0]:

#########################################################################################################################################

class DataGen(keras.utils.Sequence):
    def __init__(self, ids, path, batch_size=8, image_size=128,shuffle=True):
        self.ids = ids
        self.path = path
        self.batch_size = batch_size
        self.image_size = image_size
        self.shuffle = shuffle
        self.on_epoch_end()
        
    def __load__(self, id_name):
        
        ## Path
        y_path = os.path.join(self.path, id_name,"Y","pollutionMap") + ".png"
        x_path = os.path.join(self.path, id_name,"X/")
        all_x_files_names = sorted(os.listdir(x_path))
        ## Reading Image
        image_group_x=np.zeros((self.image_size,self.image_size,len(all_x_files_names)))

        ## Reading Image
        for i,file_name in enumerate(all_x_files_names):
            
            path_x_file_name=os.path.join(x_path,file_name)
            image_x= cv2.imread(path_x_file_name,cv2.IMREAD_GRAYSCALE)
            image_x= cv2.resize(image_x,(self.image_size, self.image_size))
            ## Normalazing
            image_x=image_x/255.0
            image_group_x[:,:,i]=image_x
        
        
        ## Reading Image
        image_y=cv2.imread(y_path,cv2.IMREAD_GRAYSCALE)
        image_y=cv2.resize(image_y,(self.image_size,self.image_size))
        ## Normalazing
        image_y = image_y/255.0
        
        image_y=np.reshape(image_y,(self.image_size,self.image_size,1))
        
        print("image_x_group",np.shape(image_group_x))
        print("image_x",np.shape(image_x))
        print("image_y",np.shape(image_y))
        
        return image_group_x,image_y
    
    def __getitem__(self, index):
        
        if(index+1)*self.batch_size > len(self.ids):
            self.batch_size = len(self.ids) - index*self.batch_size
        
        files_batch = self.ids[index*self.batch_size : (index+1)*self.batch_size]
        
        images_x  = []
        image_y = []
        
        for id_name in files_batch:
            _imgs_x, _img_y = self.__load__(id_name)
            images_x.append(_imgs_x)
            image_y.append(_img_y)
            
        images_x  = np.array(images_x)    
        image_y = np.array(image_y)
        
        return images_x, image_y
    
    def on_epoch_end(self):
          'Updates indexes after each epoch'
          self.indexes = np.arange(len(self.ids))
          if self.shuffle == True:
              np.random.shuffle(self.indexes)
    
    def __len__(self):
        return int(np.ceil(len(self.ids)/float(self.batch_size)))


In [0]:
#########################################################################################################################################

#########################################################################################################################################

image_size = 128
train_path = "dataset_images/train"
test_path = "dataset_images/test"
epochs = 100
batch_size = 6
## Validation Data Size
val_data_size = 12

#########################################################################################################################################

#########################################################################################################################################

## Training Ids

train_ids = next(os.walk(train_path))[1]
test_ids = next(os.walk(test_path))[1]
print(train_ids[0])

valid_ids = train_ids[:val_data_size]
train_ids = train_ids[val_data_size:]

gen = DataGen(train_ids, train_path, batch_size=batch_size, image_size=image_size)

x, y = gen.__getitem__(1)
print(x.shape, y.shape)

r = random.randint(0, len(x)-1)

In [0]:
import h5py
# save TRAINING data as hdf5 
with h5py.File(os.path.join( '/content/drive/My Drive/Air-D/Unet-5/dataset_images' , 'Training.hdf5') , 'w') as f:
    f.create_dataset('Training_X_3808', data=x)
    f.create_dataset('Training_Y_3808', data=y)
    
# save Testing data as HDF5
with h5py.File(os.path.join( '/content/drive/My Drive/Air-D/Unet-5/dataset_images', 'Testing.hdf5') , 'w') as f:
    f.create_dataset('Testing_X_', data=x_test)
    f.create_dataset('Testing_Y_', data=y_test)    

In [0]:
print(x.shape , y.shape)
print(x_test.shape , y_test.shape)
# r = random.randint(0, len(x)-1)

The dataset already contains the required splits of test and train and so let's continue to use the same split.

In [0]:
TRAIN_LENGTH = info.splits['train'].num_examples
BATCH_SIZE = 64
BUFFER_SIZE = 1000
STEPS_PER_EPOCH = TRAIN_LENGTH // BATCH_SIZE

In [0]:
train = dataset['train'].map(load_image_train, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test = dataset['test'].map(load_image_test)

In [0]:
train_dataset = train.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_dataset = test.batch(BATCH_SIZE)

Let's take a look at an image example and it's correponding mask from the dataset.

In [0]:
def display(display_list):
  plt.figure(figsize=(15, 15))

  title = ['Input Image', 'True Mask', 'Predicted Mask']

  for i in range(len(display_list)):
    plt.subplot(1, len(display_list), i+1)
    plt.title(title[i])
    plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
    plt.axis('off')
  plt.show()

In [0]:
for image, mask in train.take(1):
  sample_image, sample_mask = image, mask
display([sample_image, sample_mask])

## Define the model
The model being used here is a modified U-Net. A U-Net consists of an encoder (downsampler) and decoder (upsampler). In-order to learn robust features, and reduce the number of trainable parameters, a pretrained model can be used as the encoder. Thus, the encoder for this task will be a pretrained MobileNetV2 model, whose intermediate outputs will be used, and the decoder will be the upsample block already implemented in TensorFlow Examples in the [Pix2pix tutorial](https://github.com/tensorflow/examples/blob/master/tensorflow_examples/models/pix2pix/pix2pix.py). 

The reason to output three channels is because there are three possible labels for each pixel. Think of this as multi-classification where each pixel is being classified into three classes.

In [0]:
OUTPUT_CHANNELS = 3

As mentioned, the encoder will be a pretrained MobileNetV2 model which is prepared and ready to use in [tf.keras.applications](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/applications). The encoder consists of specific outputs from intermediate layers in the model. Note that the encoder will not be trained during the training process.

In [0]:
base_model = tf.keras.applications.MobileNetV2(input_shape=[128, 128, 3], include_top=False)

# Use the activations of these layers
layer_names = [
    'block_1_expand_relu',   # 64x64
    'block_3_expand_relu',   # 32x32
    'block_6_expand_relu',   # 16x16
    'block_13_expand_relu',  # 8x8
    'block_16_project',      # 4x4
]
layers = [base_model.get_layer(name).output for name in layer_names]

# Create the feature extraction model
down_stack = tf.keras.Model(inputs=base_model.input, outputs=layers)

down_stack.trainable = False

The decoder/upsampler is simply a series of upsample blocks implemented in TensorFlow examples.

In [0]:
up_stack = [
    pix2pix.upsample(512, 3),  # 4x4 -> 8x8
    pix2pix.upsample(256, 3),  # 8x8 -> 16x16
    pix2pix.upsample(128, 3),  # 16x16 -> 32x32
    pix2pix.upsample(64, 3),   # 32x32 -> 64x64
]

In [0]:
def unet_model(output_channels):

  # This is the last layer of the model
  last = tf.keras.layers.Conv2DTranspose(
      output_channels, 3, strides=2,
      padding='same', activation='softmax')  #64x64 -> 128x128

  inputs = tf.keras.layers.Input(shape=[128, 128, 3])
  x = inputs

  # Downsampling through the model
  skips = down_stack(x)
  x = skips[-1]
  skips = reversed(skips[:-1])

  # Upsampling and establishing the skip connections
  for up, skip in zip(up_stack, skips):
    x = up(x)
    concat = tf.keras.layers.Concatenate()
    x = concat([x, skip])

  x = last(x)

  return tf.keras.Model(inputs=inputs, outputs=x)

## Train the model
Now, all that is left to do is to compile and train the model. The loss being used here is losses.sparse_categorical_crossentropy. The reason to use this loss function is because the network is trying to assign each pixel a label, just like multi-class prediction. In the true segmentation mask, each pixel has either a {0,1,2}. The network here is outputting three channels. Essentially, each channel is trying to learn to predict a class, and losses.sparse_categorical_crossentropy is the recommended loss for such a scenario. Using the output of the network, the label assigned to the pixel is the channel with the highest value. This is what the create_mask function is doing.

In [0]:
model = unet_model(OUTPUT_CHANNELS)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Have a quick look at the resulting model architecture:

In [0]:
tf.keras.utils.plot_model(model, show_shapes=True)

Let's try out the model to see what it predicts before training.

In [0]:
def create_mask(pred_mask):
  pred_mask = tf.argmax(pred_mask, axis=-1)
  pred_mask = pred_mask[..., tf.newaxis]
  return pred_mask[0]

In [0]:
def show_predictions(dataset=None, num=1):
  if dataset:
    for image, mask in dataset.take(num):
      pred_mask = model.predict(image)
      display([image[0], mask[0], create_mask(pred_mask)])
  else:
    display([sample_image, sample_mask,
             create_mask(model.predict(sample_image[tf.newaxis, ...]))])

In [0]:
show_predictions()

Let's observe how the model improves while it is training. To accomplish this task, a callback function is defined below. 

In [0]:
class DisplayCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    clear_output(wait=True)
    show_predictions()
    print ('\nSample Prediction after epoch {}\n'.format(epoch+1))

In [0]:
EPOCHS = 20
VAL_SUBSPLITS = 5
VALIDATION_STEPS = info.splits['test'].num_examples//BATCH_SIZE//VAL_SUBSPLITS

model_history = model.fit(train_dataset, epochs=EPOCHS,
                          steps_per_epoch=STEPS_PER_EPOCH,
                          validation_steps=VALIDATION_STEPS,
                          validation_data=test_dataset,
                          callbacks=[DisplayCallback()])

In [0]:
loss = model_history.history['loss']
val_loss = model_history.history['val_loss']

epochs = range(EPOCHS)

plt.figure()
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'bo', label='Validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss Value')
plt.ylim([0, 1])
plt.legend()
plt.show()

## Make predictions

Let's make some predictions. In the interest of saving time, the number of epochs was kept small, but you may set this higher to achieve more accurate results.

In [0]:
show_predictions(test_dataset, 3)

## Next steps
Now that you have an understanding of what image segmentation is and how it works, you can try this tutorial out with different intermediate layer outputs, or even different pretrained model. You may also challenge yourself by trying out the [Carvana](https://www.kaggle.com/c/carvana-image-masking-challenge/overview) image masking challenge hosted on Kaggle.

You may also want to see the [Tensorflow Object Detection API](https://github.com/tensorflow/models/tree/master/research/object_detection) for another model you can retrain on your own data.